In [9]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [11]:
import os
family_group = "3.Satyrine"
annotated_genome_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/9.CLOCK Exon Analysis/{family_group}/1.Blast_result"
species_list = os.listdir(annotated_genome_location)
if "desktop.ini" in species_list:
    species_list.remove("desktop.ini")
# species_list = ["Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Limenitis_camilla"]
# species_list = ["Argynnis_bischoffii_washingtonia","Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Bicyclus_anynana","Coenonympha_glycerion","Elymnias_hypermnestra","Erebia_aethiops","Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Vanessa_tameamea"]
for species in species_list:
    print(species)
    blast_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/9.CLOCK Exon Analysis/{family_group}/1.Blast_result"
    
    list_of_files = os.listdir(f"{blast_location}/{species}")
    query_species_list = []
    coordinate_detail_dictionary = {}
    coordinate_info_dictionary = {}
    scaffold = ''
    for file in list_of_files:
        if file.endswith(".csv") and len(file.split("_")) >= 5 and not(file.endswith("_old.csv")) and not(file.endswith("_old_backup.csv")):
#             print(file)
            if "." in file[:-4]:
                query_species = f"{file.split('_')[-2].split('.')[1]}_{file.split('_')[-1].split('.')[0]}"
            else:
                query_species = f"{file.split('_')[-2]}_{file.split('_')[-1].split('.')[0]}"
            query_species_list.append(query_species)
            coordinate_detail_dictionary[query_species] = {}
            with open(f"{blast_location}/{species}/{file}", 'r') as coordinate_file_open:
                coordinate_file_lines_list  =  coordinate_file_open.readlines()
            if scaffold == '':
                scaffold = coordinate_file_lines_list[1].split(",")[1]
    
            for i in range(1,len(coordinate_file_lines_list)):
                
                coordinate_file_lines_split = coordinate_file_lines_list[i].split(",")
                start,stop = coordinate_file_lines_split[2],coordinate_file_lines_split[3]
                exon = f"Exon_{i}"
                if exon not in coordinate_info_dictionary:
                    coordinate_info_dictionary[exon] = {}
                coordinate_info_dictionary[exon][query_species] = [start,stop]
                coordinate_detail_dictionary[query_species][exon] = coordinate_file_lines_list[i]
            
            # break
            # print(query_species)
            # print(file)
#     print(coordinate_info_dictionary)
#     print(coordinate_detail_dictionary)
    # assert False
    output = coordinate_file_lines_list[0]
    for exon,species_specific_coordinates in coordinate_info_dictionary.items():
        # print(exon,species_specific_coordinates)
        coordinates_list = []
        previous_coordinates = ""
        match = 1
        for query_species, coordinates in coordinate_info_dictionary[exon].items():
            coordinates_list.append([query_species,coordinates])
            if previous_coordinates != '':
                if coordinates != previous_coordinates:
                    match = 0
                    
            previous_coordinates = coordinates
       
        
        
        if match == 0:
            print(exon,"mismatch")
            print(coordinates_list)
                
            while True:
                choice = input("Proceed?")
                if choice[0].lower() == "y" or choice[0].lower() == "n":
                    break
            
            while True:
                if choice[0].lower() == "y":
                    alignment_detail_dictionary = {}
                    for query_species_name in query_species_list:
                        list_of_folders  = os.listdir(f"{blast_location}/{species}")
                        for folders in list_of_folders:
                            if folders.endswith(query_species_name):
                                # print()
                                
                                with open(f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query.txt", 'r') as query_file_open:
                                    # alignment_detail = query_file_open.readlines()[0]
                                    alignment_detail = query_file_open.readlines()[0].split("set")[1][:-1]
                                    alignment_detail_dictionary[query_species_name] = alignment_detail
                                with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/{exon}_translated_genomic_sequence_{alignment_detail}.fa.hat2", 'r') as alignment_score_file:
                                    distance_score = float(alignment_score_file.readlines()[-1].rstrip())
                                try:
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query_spliced.txt", 'r') as new_query_file:
                                        new_query = (new_query_file.readlines()[-1].rstrip())
                                except:
                                    new_query = "Issue with query"
                                print(query_species_name,exon, alignment_detail, distance_score,f"\n{new_query}")
                    while True:            
                        check_alignment = input("Check Alignment??")
                        if check_alignment[0].lower() == "y" or check_alignment[0].lower() == "n":
                            break
                    if check_alignment[0].lower() == "y":
                        for query_species_name in query_species_list:
                            list_of_folders  = os.listdir(f"{blast_location}/{species}")
                            for folders in list_of_folders:
                                if folders.endswith(query_species_name):
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/alignment_clustal_{exon}_translated_genomic_sequence_{alignment_detail_dictionary[query_species_name]}.fa.txt", 'r') as clustal_alignment_file:
                                        clustal_alignment = clustal_alignment_file.readlines()
                                    print("".join(clustal_alignment))
                                    
                                # assert False
                    while True:
                        try:
                            species_number = (input(f"Choose Species 1 - {len(query_species_list)}"))
                            
                            
                            if species_number[0].lower() == 'n':
                                output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                                break 
                            output += coordinate_detail_dictionary[query_species_list[int(species_number)-1]][exon]
                            break
                        except:
                            print("Retry")
                    break
                elif choice[0].lower() == "n":
                    output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                    break
                
                
        else:
            print(exon, "Match")
            print(coordinates_list)
            output += coordinate_detail_dictionary[query_species_list[0]][exon]
        
       

        # assert False
    # print(coordinate_detail_dictionary)
    print(output)
    with open(f"{blast_location}/{species}/final_coordinates.csv", "w") as out_file:
        out_file.write(output)
    
    # 'Coenonympha_glycerion,OY979626.1,7015875,7015963,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_0_rightoh_2_query_Exon_1,1,29,29\n'

Bicyclus_anynana
Exon_1 Match
[['Pararge_aegeria', ['8762140', '8762288']], ['Bicyclus_anynana', ['8762140', '8762288']], ['Maniola_jurtina', ['8762140', '8762288']]]
Exon_2 Match
[['Pararge_aegeria', ['8758140', '8758288']], ['Bicyclus_anynana', ['8758140', '8758288']], ['Maniola_jurtina', ['8758140', '8758288']]]
Exon_3 Match
[['Pararge_aegeria', ['8757713', '8757894']], ['Bicyclus_anynana', ['8757713', '8757894']], ['Maniola_jurtina', ['8757713', '8757894']]]
Exon_4 Match
[['Pararge_aegeria', ['8757213', '8757330']], ['Bicyclus_anynana', ['8757213', '8757330']], ['Maniola_jurtina', ['8757213', '8757330']]]
Exon_5 Match
[['Pararge_aegeria', ['8753106', '8753201']], ['Bicyclus_anynana', ['8753106', '8753201']], ['Maniola_jurtina', ['8753106', '8753201']]]
Exon_6 Match
[['Pararge_aegeria', ['8751939', '8751999']], ['Bicyclus_anynana', ['8751939', '8751999']], ['Maniola_jurtina', ['8751939', '8751999']]]
Exon_7 Match
[['Pararge_aegeria', ['8748753', '8748991']], ['Bicyclus_anynana', ['8

Proceed?y
Pararge_aegeria Exon_12 41_frame0 0.342 
WLWQEQLQRTHRELQELIARQQEELRQVKEQLLLARLGILQPIIN
Bicyclus_anynana Exon_12 41_frame0 0.378 
EQLQRTHRELQELIARQQEELRQVKEQLLLARLGILQPIIN
Maniola_jurtina Exon_12 41_frame0 0.1 
EQLQRTHRELQELIARQQEELRQVKEQLLLARLGILQPIIN
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Pararge_aegeria ---QVQMQHTHRELQHLIARQQEELRQVKEQLMLARLGIFQPVIS
set41_frame0    WLWQEQLQRTHRELQELIARQQEELRQVKEQLLLARLGILQPIIN
                   * *:*:******.****************:******:**:*.

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Bicyclus_anynan ----EQLQRTHRELQHMIFRQQEELRLVKEQLMLARLGIFQPMIS
set41_frame0    WLWQEQLQRTHRELQELIARQQEELRQVKEQLLLARLGILQPIIN
                    ***********.:* ******* *****:******:**:*.

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Maniola_jurtina ----EQLQRTHRELQQMIVRQQEELRQVKEQLLLARLGILQPIIN
set41_frame0    WLWQEQLQRTHRELQELIARQQEELRQVKEQLLLARLGILQPIIN
                    ***********::*.**************************


Exon_1 Match
[['Pararge_aegeria', ['8496961', '8497109']], ['Bicyclus_anynana', ['8496961', '8497109']], ['Maniola_jurtina', ['8496961', '8497109']]]
Exon_2 Match
[['Pararge_aegeria', ['8492961', '8493109']], ['Bicyclus_anynana', ['8492961', '8493109']], ['Maniola_jurtina', ['8492961', '8493109']]]
Exon_3 Match
[['Pararge_aegeria', ['8492139', '8492320']], ['Bicyclus_anynana', ['8492139', '8492320']], ['Maniola_jurtina', ['8492139', '8492320']]]
Exon_4 Match
[['Pararge_aegeria', ['8491523', '8491640']], ['Bicyclus_anynana', ['8491523', '8491640']], ['Maniola_jurtina', ['8491523', '8491640']]]
Exon_5 Match
[['Pararge_aegeria', ['8489614', '8489709']], ['Bicyclus_anynana', ['8489614', '8489709']], ['Maniola_jurtina', ['8489614', '8489709']]]
Exon_6 Match
[['Pararge_aegeria', ['8489309', '8489366']], ['Bicyclus_anynana', ['8489309', '8489366']], ['Maniola_jurtina', ['8489309', '8489366']]]
Exon_7 Match
[['Pararge_aegeria', ['8488382', '8488620']], ['Bicyclus_anynana', ['8488382', '8488620

Proceed?y
Pararge_aegeria Exon_12 65_frame1 0.427 
FAPLAVETLLHLRGWFCNDDASLWPWQEQLQRTHRELQQMIMRQQEELRQVKEQLLLARFGILQPIIN
Bicyclus_anynana Exon_12 65_frame1 0.354 
EQLQRTHRELQQMIMRQQEELRQVKEQLLLARFGILQPIIN
Maniola_jurtina Exon_12 65_frame1 0.061 
EQLQRTHRELQQMIMRQQEELRQVKEQLLLARFGILQPIIN
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Pararge_aegeria --------------------------QVQMQHTHRELQHLIARQQEELRQVKEQLMLARL
set65_frame1    FAPLAVETLLHLRGWFCNDDASLWPWQEQLQRTHRELQQMIMRQQEELRQVKEQLLLARF
                                          * *:*:******::* *************:***:

Pararge_aegeria GIFQPVIS
set65_frame1    GILQPIIN
                **:**:*.

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Bicyclus_anynan ---------------------------EQLQRTHRELQHMIFRQQEELRLVKEQLMLARL
set65_frame1    FAPLAVETLLHLRGWFCNDDASLWPWQEQLQRTHRELQQMIMRQQEELRQVKEQLLLARF
                                           ***********:**:******* *****:***:

Bicyclus_anynan GIFQPMIS
set65_frame1    GILQPII

Exon_1 Match
[['Pararge_aegeria', ['16302870', '16303018']], ['Bicyclus_anynana', ['16302870', '16303018']], ['Maniola_jurtina', ['16302870', '16303018']]]
Exon_2 Match
[['Pararge_aegeria', ['16306870', '16307018']], ['Bicyclus_anynana', ['16306870', '16307018']], ['Maniola_jurtina', ['16306870', '16307018']]]
Exon_3 Match
[['Pararge_aegeria', ['16307938', '16308119']], ['Bicyclus_anynana', ['16307938', '16308119']], ['Maniola_jurtina', ['16307938', '16308119']]]
Exon_4 Match
[['Pararge_aegeria', ['16308256', '16308373']], ['Bicyclus_anynana', ['16308256', '16308373']], ['Maniola_jurtina', ['16308256', '16308373']]]
Exon_5 Match
[['Pararge_aegeria', ['16309617', '16309712']], ['Bicyclus_anynana', ['16309617', '16309712']], ['Maniola_jurtina', ['16309617', '16309712']]]
Exon_6 Match
[['Pararge_aegeria', ['16310114', '16310174']], ['Bicyclus_anynana', ['16310114', '16310174']], ['Maniola_jurtina', ['16310114', '16310174']]]
Exon_7 Match
[['Pararge_aegeria', ['16310958', '16311196']], ['B

Proceed?y
Pararge_aegeria Exon_9 96_frame0 1.081 
YADVDTSTKQENTEQDTRSEADPNQSGLKEPSPEAAVAVSPSYMSEASEAFGNSYHSLSQ
Bicyclus_anynana Exon_9 96_frame0 0.75 
YADVDTSTKQENTEQDTRSEADPNQSGLKEPSPEAAVAVSPSYMSEASEAFGNSYHSLSQVRIALFETNKICLFASLERATHDSF
Maniola_jurtina Exon_9 96_frame0 0.207 
YADVDTSTKQENTEQDTRSEADPNQSGLKEPSPEAAVAVSPSYMSEASEAFGNSYHSLSQ
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Pararge_aegeria --------YVDVDTPTKEENIEEEEAVSPEPCHSRVKEQSPVSSAAVSPVYISEPGEPYG
set96_frame0    FVVMTSCSYADVDTSTKQENTEQDTRSEADPNQSGLKEPSPEAAVAVSPSYMSEASEAFG
                        *.****.**:** *::   ..:* :* :** ** ::.**** *:**..*.:*

Pararge_aegeria SS-------------------------------------QSPSR----------------
set96_frame0    NSYHSLSQVRIALFETNKICLFASLERATHDSFVRRPHLPSPFHQLSSFLIFSCSQNKYV
                .*                                      ** :                

Pararge_aegeria -
set96_frame0    I
                 

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Bicyclus_anynan --

Proceed?y
Pararge_aegeria Exon_9 22_frame0 1.109 
YADVDTSTKQENTEQDTRSEADPIQPGLKEPSPEVIAAVSPSYMSEASEAFGTSYHSLSQ
Bicyclus_anynana Exon_9 22_frame0 0.871 
YADVDTSTKQENTEQDTRSEADPIQPGLKEPSPEVIAAVSPSYMSEASEAFGTSYHSLSQVRNALFKSN
Maniola_jurtina Exon_9 22_frame0 0.0 
YADVDTSTKQENTEQDTRSEADPIQPGLKEPSPEVIAAVSPSYMSEASEAFGTSYHSLSQ
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Pararge_aegeria --------YVDVDTPTKEENIEEEEAVSPEPCHSRVKEQSPVSSAAVSPVYISEPGEPYG
set22_frame0    IIVMISCSYADVDTSTKQENTEQDTRSEADPIQPGLKEPSPEVIAAVSPSYMSEASEAFG
                        *.****.**:** *::   ..:* :. :** **   ***** *:**..*.:*

Pararge_aegeria SS-QSPSR----------------
set22_frame0    TSYHSLSQVRNALFKSNVLPVCWI
                :* :* *:                

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Bicyclus_anynan --------YVDVQTPTKQENIEQDTSTDCEPSHSGLKEPLSEEATAVSPSYMSESSDAYG
set22_frame0    IIVMISCSYADVDTSTKQENTEQDTRSEADPIQPGLKEPSPEVIAAVSPSYMSEASEAFG
                        *.**:*.***** **** ::

Proceed?y
Pararge_aegeria Exon_12 148_frame1 0.39 
PWQEQLQRTHRELQQMIVRQQEELRQVKEQLLLARLGILQPIIN
Bicyclus_anynana Exon_12 148_frame1 0.33 
EQLQRTHRELQQMIVRQQEELRQVKEQLLLARLGILQPIIN
Maniola_jurtina Exon_12 148_frame1 0.0 
EQLQRTHRELQQMIVRQQEELRQVKEQLLLARLGILQPIIN
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Pararge_aegeria --QVQMQHTHRELQHLIARQQEELRQVKEQLMLARLGIFQPVIS
set148_frame1   PWQEQLQRTHRELQQMIVRQQEELRQVKEQLLLARLGILQPIIN
                  * *:*:******::*.*************:******:**:*.

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Bicyclus_anynan ---EQLQRTHRELQHMIFRQQEELRLVKEQLMLARLGIFQPMIS
set148_frame1   PWQEQLQRTHRELQQMIVRQQEELRQVKEQLLLARLGILQPIIN
                   ***********:**.******* *****:******:**:*.

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Maniola_jurtina ---EQLQRTHRELQQMIVRQQEELRQVKEQLLLARLGILQPIIN
set148_frame1   PWQEQLQRTHRELQQMIVRQQEELRQVKEQLLLARLGILQPIIN
                   *****************************************

Choose S

Proceed?y
Pararge_aegeria Exon_12 207_frame0 0.39 
GMNDDASWWPWQEQLQRTHRELQQMIVRQQEELRQVKEQLLLARLGILQPIIN
Bicyclus_anynana Exon_12 207_frame0 0.33 
EQLQRTHRELQQMIVRQQEELRQVKEQLLLARLGILQPIIN
Maniola_jurtina Exon_12 207_frame0 0.0 
EQLQRTHRELQQMIVRQQEELRQVKEQLLLARLGILQPIIN
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Pararge_aegeria -----------QVQMQHTHRELQHLIARQQEELRQVKEQLMLARLGIFQPVIS
set207_frame0   GMNDDASWWPWQEQLQRTHRELQQMIVRQQEELRQVKEQLLLARLGILQPIIN
                           * *:*:******::*.*************:******:**:*.

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Bicyclus_anynan ------------EQLQRTHRELQHMIFRQQEELRLVKEQLMLARLGIFQPMIS
set207_frame0   GMNDDASWWPWQEQLQRTHRELQQMIVRQQEELRQVKEQLLLARLGILQPIIN
                            ***********:**.******* *****:******:**:*.

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Maniola_jurtina ------------EQLQRTHRELQQMIVRQQEELRQVKEQLLLARLGILQPIIN
set207_frame0   GMNDDASWWPWQEQLQRTHRELQQMIVRQQEELRQVKEQLLLA